In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys
sys.path.append('../')

In [25]:
def get_base_prefix_compat():
    """Get base/real prefix, or sys.prefix if there is none."""
    return getattr(sys, "base_prefix", None) or getattr(sys, "real_prefix", None) or sys.prefix

def in_virtualenv():
    return get_base_prefix_compat() != sys.prefix

In [26]:
if not in_virtualenv():
    print('This script must be run from within the local virtual environment')
    print(f'try:\npipenv run python3 {sys.argv[0]}')
    sys.exit()

In [27]:
from paperpi.library import Plugin, CacheFiles, get_help
from importlib import import_module
from pathlib import Path
import paperpi.my_constants as paperpi_constants
import logging
from IPython.display import Image 
import argparse
import sys
import re


In [28]:
logging.basicConfig(level='INFO')

In [29]:
logger = logging.getLogger(__name__)

In [30]:
def setup_plugins(project_root, plugin_list=None, resolution=(640, 400), skip_layouts=False):
    '''create dictionary of plugins using sample configurations provided in each plugin directory
    
    Args:
        project_root(`str`): directory where main script is located
        use_all_layouts(`Bool`): use all discovered layouts
        plugin_list(`list`): list of plugins to update; when None all discovered plugins will be updated
        resolution(`tuple` of `int`): screen resolution to use when generating images
        load_layouts(`bool`): when True, create a full Plugin object with a layout and an image
        
    Returns:
        `dict` of Plugin objects and layout name used to generate the Plugin
        '''


    def font_path(layout):
        '''add font path to layout'''
        for k, block in layout.items():
            font = block.get('font', None)
            if font:
                font = font.format(paperpi_constants.FONTS)
                block['font'] = font
        return layout
    
    plugin_path = Path(f'{project_root}/{paperpi_constants.PLUGINS}')
    
    # discover all plugins
    plugins = get_help._get_modules(plugin_path)
    logger.debug(f'discovered plugins: {plugins}')
    
    # reduce the list down to just the specified plugins
    if plugin_list:
        my_list = []
        for i in plugins:
            if i in plugin_list:
                my_list.append(i)
            else:
                print(f'specified plugin not found: {i}')
        plugins = my_list
    
    cache = CacheFiles()
    
    plugin_dict = {}
    
    
    # get the base name for the plugins package
    pkg_name = '.'.join([i for i in plugin_path.parts if not i.startswith('.')])

    for plugin in plugins:
        plugin_dict[plugin] = []
        all_layouts = {}
        print(f'found plugin {plugin}')
#         pkg_name = f'{".".join(plugin_path.parts)}'
        logger.debug(f'importing pkg: {pkg_name}')
        
        # import the plugin, layout and sample configuration
        try:
            module = import_module(f'{pkg_name}.{plugin}')            
            layout_import =  import_module(f'{pkg_name}.{plugin}.layout')
            sample_import = import_module(f'{pkg_name}.{plugin}.sample')
        except ModuleNotFoundError as e:
            print(f'skipping plugin {plugin} due to previous error: {e}')
            continue
        
        # get all of the layout dictionarys from the layout file
        for a in dir(layout_import):
            if not a.startswith('_') and isinstance(getattr(layout_import, a), dict):
                all_layouts[a] = (getattr(layout_import, a))
                
        if len(all_layouts) < 1:
            print(f'skipping plugin {plugin} due to missing layouts')
            continue
            
        # remove the default layout from the list if there are multiple layouts defined
#         if len(all_layouts) > 1 and 'layout' in all_layouts:
#             all_layouts.pop('layout')
        
        # make sure there is a valid configuration:
        try:
            config = sample_import.config
        except AttributeError as e:
            print(f'skipping plugin {plugin} due to missing sample configuration: {e}')
            continue
        
        # setup plugin
        if skip_layouts:
#             print('skipping creating full layouts and creating images due to command line switch (-s)')
            continue
        else:
            for name, layout in all_layouts.items():
                print(f'adding layout: {name}')
                layout = font_path(layout)
                if not skip_layouts:
                    my_plugin = Plugin(resolution=resolution,
                                       cache=cache,
                                       layout=layout,
                                       update_function=module.update_function,
                                       config=config
                                      )
                    my_plugin.refresh_rate = 1

                    try:
                        if 'kwargs' in config:
                            my_plugin.update(**config['kwargs'])

                        else:
                            my_plugin.update()
                    except Exception as e:
                        print(f'plugin "{plugin}" could not be configured due to errors: {e}')
                else:
                    my_plugin = None
                plugin_dict[plugin].append({
                                       'plugin': plugin,
                                       'module': module, 
                                       'plugin_obj': my_plugin, 
                                       'doc_path': plugin_path/plugin,
                                       'layout': name})
    return plugin_dict


In [31]:
def update_ini_file(plugin_dict):
    '''append sample configurations for each module to the default paperpi.ini file
    distributed at install'''
    
    base_ini_file = '../install/paperpi_base.ini'
    output_ini_file = '../paperpi/config/paperpi.ini'
    
    config_sections = []
    skipped_sections = []
    
    print(f'updating {output_ini_file} using sample configs from plugins')
    
    
    for plugin, data, in plugin_dict.items():
        try:
            sample_config = data[0]['module'].constants.sample_config
        except (IndexError, AttributeError):
            logger.info(f'no valid module data for plugin {plugin}; add {plugin}/constants.sample_config')
            skipped_sections.append(plugin)
            continue
                        
        # check that it matches the format
        match = re.match('^\s{0,}\[Plugin', sample_config)
        try:
            # set the plugin to be disabled by default
            if match.string:
                sample_config = re.sub('^\s{0,}\[Plugin', '[xPlugin', sample_config)
            else:
                print(f'plugin {plugin} does not have a standard sample_config string. Please check formatting.')
                continue
        except AttributeError:
            print(f'plugin {plugin} has no valid sample_config string: {sample_config}')
            continue
        
        config_sections.append(sample_config)
        config_sections.append('\n')
    
    output_ini_list = []
    with open(base_ini_file, 'r') as base_f:
        for i in base_f:
            output_ini_list.append(i)
    
    output_ini_list.extend(config_sections)
    
    print(f'writing updated .ini file to {output_ini_file}')
    with open(output_ini_file, 'w') as out_f:
        for i in output_ini_list:
            out_f.write(i)
            
    if len(skipped_sections) > 0:
        print('\nThese plugins did not have valid sample configurations:')
        print(skipped_sections)

In [32]:
def create_readme(plugin_dict, project_root, overwrite_images=False):
    plugin_docs = {}
    plugin_path = Path(f'{project_root}/{paperpi_constants.PLUGINS}')
    readme_name = 'README'
    readme_additional = '_additional'
    readme_suffix = '.md'
    
    # run through each plugin and gather the document information
    for plugin, layouts in plugin_dict.items():
        print(f'Processing plugins: {plugin}')
        try:
            doc_path = Path(layouts[0]['doc_path'])
            plugin_readme = Path(doc_path)/f'{readme_name}{readme_suffix}'
            plugin_additional_readme = Path(doc_path)/f'{readme_name}{readme_additional}{readme_suffix}'
        except IndexError:
            print(f'   skipping {plugin} due to incomplete documentation')
            continue
        
        # pull the help information for the plugin
        readme_text = get_help.get_help(plugin, False, plugin_path=plugin_path)
        
        # read the additional documentation
        if plugin_additional_readme.exists():
            with open(plugin_additional_readme, 'r') as file:
                additional_text = file.read()
        else:
            additional_text = '' 
        
        default_layout_image = {'filename': 'not found',
                                'path': 'none',
                                'layout_name': 'layout'}
        layout_images = []
        # gather all the images associated with the plugin
        for layout in layouts:
            layout_name = layout['layout']
            image_filename = f'{plugin}.{layout["layout"]}-sample.png'
            image_path = Path(doc_path)/image_filename
                        
            print(f'   processing layout: {layout_name}')
            try:
                image = layout['plugin_obj'].image
            except AttributeError:
                print('   - no image available, skipping this layout')
                continue
                
            if (image_path.exists() and overwrite_images) or not image_path.exists():
                print(f'   saving image: {image_path}')
                image.save(image_path)
            else:
                print(f'   image exists, skipping save')
                
            
            
            layout_entry = {'filename': image_filename,
                            'path': image_path,
                            'layout_name': layout_name}
            
            # try to locate the default "layout" to show at the top of the documentation
            if layout_name == 'layout':
                default_layout_image.update(layout_entry)
            
            layout_images.append(layout_entry)

        print('   writing README files')
        with open(plugin_readme, 'w') as file:
            plugin_name = ".".join(doc_path.parts)
            file.write(f'# {plugin}\n')
            file.write(f'![sample image for plugin {plugin_name}](./{default_layout_image["filename"]}) \n\n')
            file.write(f'```\n{readme_text}\n```\n\n')
            file.write(f'## Provided Layouts:\n\n')
            for l in layout_images:
                file.write(f'layout: **{l["layout_name"]}**\n\n')  
                file.write(f'![sample image for plugin {l["layout_name"]}](./{l["filename"]}) \n\n\n')

            file.write(additional_text)
        
        plugin_docs[plugin] = {'readme': plugin_readme, 'image': default_layout_image["path"]}
            
    
    return plugin_docs

In [33]:
def update_plugin_docs(plugin_docs, doc_path):
    '''update Plugin.md documentation with snips from all plugin READMEs
    
    Args:
        plugin_docs(dict): dictionary provided by create_readme'''
    doc_path = Path(doc_path)
    plugin_readme_source = Path(doc_path/'source/Plugins.md')
    plugin_readme_post_source = Path(doc_path/'source/Plugins_post.md')
    plugin_readme_final = Path(doc_path/plugin_readme_source.name)
    print('updating Plugins.md...')
    print(f'using: {plugin_readme_source}')
    print(f'using postscript file: {plugin_readme_post_source}')
    with open(plugin_readme_source, 'r') as file:
        source = file.read()
        
    with open(plugin_readme_post_source, 'r') as file:
        post = file.read()
    
    with open(plugin_readme_final, 'w') as file:
        file.write(source)
        for plugin, values in plugin_docs.items():
            file.write(f'### [{plugin}]({values["readme"]})\n')
            file.write(f'![{plugin} sample Image]({values["image"]})\n\n')
            
        file.write(post)        

In [34]:
# update_plugin_docs(plugin_d, '../documentation')

In [37]:
def main():
    

        
    parser = argparse.ArgumentParser(description='create_docs')
 
    parser.add_argument('-o', '--overwrite_images', default=False, action='store_true',
                       help='overwrite existing images for plugins when updating README files')
    
    parser.add_argument('-p', '--plugin_list', default=None, nargs='*', 
                       help='list of specific plugins to process')
    
    parser.add_argument('-r', '--project_root', default='../paperpi/', nargs=1,
                       help='path to project root')
    
    parser.add_argument('-d', '--documentation_path', default='../documentation',
                       help='path to documentation directory')
    
    parser.add_argument('--log_level', default='INFO', help='set logging output level')
    
#     parser.add_argument('-s' '--skip_layouts', default=False, action='store_true',
#                         help='skip creating layouts for each plugin')
    
# this breaks README creation
    parser.add_argument('-s', '--skip_layouts', default=False, action='store_true',
                        help='skip loading layouts and do not create images')
    
    
    args = parser.parse_args()
    
    logger.root.setLevel(args.log_level)
    
#     args = parser.parse_known_args()
    plugin_dict = setup_plugins(args.project_root, args.plugin_list, skip_layouts=args.skip_layouts)
    
    if args.skip_layouts:
        print('Documentation will not be updated due to option "--skip_layouts"')
        update_ini_file(plugin_dict)        
    else:
        plugin_docs = create_readme(plugin_dict, 
                                    project_root=args.project_root,
                                    overwrite_images=args.overwrite_images,)
    
        update_plugin_docs(plugin_docs, doc_path=args.documentation_path)
    
    return plugin_dict
    

In [36]:
if __name__ == "__main__":
    if '-f' in sys.argv:
        logger.debug('looks like this is running in a Jupyter notebook')
        idx = sys.argv.index('-f')
        del sys.argv[idx:idx+2]    
    print('updating documents...')
    r = main()

INFO:root:section: [.............text.............]
INFO:root:section: [.........attribution..........]
INFO:root:section: [.............time.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [_____________text_____________]
INFO:root:section: [_________attribution__________]
INFO:root:section: [_____________time_____________]
INFO:root:the text will spill outside of padded area using these values
INFO:root:update function for reddit_quote


updating documents...
found plugin reddit_quote
adding layout: layout


INFO:root:caching data
INFO:root:section: [.............text.............]
INFO:root:section: [.........attribution..........]
INFO:root:section: [.............time.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [_____________text_____________]
INFO:root:section: [_________attribution__________]
INFO:root:section: [_____________time_____________]
INFO:root:the text will spill outside of padded area using these values
INFO:root:update function for reddit_quote
INFO:root:section: [.............text.............]
INFO:root:section: [.........attribution..........]
INFO:root:section: [.............time.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [_____________text_____________]
INFO:root:section: [_________attribution__________]
INFO:root:section: [_____________time_____________]
INFO:root:the text will spill outside of padded area using these values
INFO:root:update function for reddit_quote
INFO:root:section: [.............text............

adding layout: quote
adding layout: quote_inverse
adding layout: quote_small_screen


INFO:root:section: [_____________text_____________]
INFO:root:section: [_________attribution__________]
INFO:root:update function for reddit_quote
INFO:root:section: [...........coverart...........]
INFO:root:section: [............artist............]
INFO:root:section: [............album.............]
INFO:root:section: [............title.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [___________coverart___________]
INFO:root:section: [____________artist____________]


...

adding layout: ticker_simple


INFO:paperpi.plugins.crypto.crypto:caching data from feed
INFO:paperpi.plugins.crypto.crypto:using cached image: /tmp/pen5_ack/None_bitcoin_v_usd/bitcoin.png
INFO:root:the text will spill outside of padded area using these values
INFO:root:the text will spill outside of padded area using these values
INFO:root:the text will spill outside of padded area using these values
INFO:root:section: [...........bin_img............]
INFO:root:section: [.............time.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [___________bin_img____________]
INFO:root:section: [_____________time_____________]


found plugin dec_binary_clock
adding layout: dec_binary_clock


INFO:root:section: [...........bin_img............]
INFO:root:section: [.............time.............]
INFO:root:[[____setting blocks____]]
INFO:root:section: [___________bin_img____________]
INFO:root:section: [_____________time_____________]


adding layout: layout
Processing plugins: reddit_quote
   processing layout: layout
   image exists, skipping save
   processing layout: quote
   image exists, skipping save
   processing layout: quote_inverse
   image exists, skipping save
   processing layout: quote_small_screen
   image exists, skipping save
   writing README files
Processing plugins: librespot_client
   processing layout: layout
   image exists, skipping save
   processing layout: three_rows_text_only
   image exists, skipping save
   processing layout: two_column_three_row
   image exists, skipping save
   processing layout: two_rows_text_only
   image exists, skipping save
   writing README files
Processing plugins: moon_phase
   processing layout: layout
   image exists, skipping save
   processing layout: moon_data
   image exists, skipping save
   processing layout: moon_only
   image exists, skipping save
   writing README files
Processing plugins: lms_client
   processing layout: layout
   image exists, skip

In [21]:
!jupyter-nbconvert --to python --template python_clean create_docs.ipynb

[NbConvertApp] Converting notebook create_docs.ipynb to python
[NbConvertApp] Writing 14374 bytes to create_docs.py
